In [ ]:
import requests
import json
import httpx
from typing import Optional, Dict, Any
import asyncio
from pprint import pprint
from src.URESTAdapter import URESTAdapter
from src.utils.OASParser import OASParser
from kafka import KafkaProducer


Инициализация Продюссера

In [ ]:
BOOTSTRAP_SERVERS = ['localhost:9092']  # или ваш сервер Kafka
producer = KafkaProducer(
    bootstrap_servers=BOOTSTRAP_SERVERS,
    value_serializer=lambda v: json.dumps(v).encode('utf-8'),
    # Дополнительные параметры:
    acks='all',  # подтверждение от всех реплик
    retries=3,    # количество попыток
    max_in_flight_requests_per_connection=1,
    compression_type='gzip'  # сжатие
)


In [ ]:
Определяем сущность (пока временно это operationID),он же будет топиком в кафку

In [ ]:
TOPIC_NAME = 'suggestions_dadata_suggestBank'
#  forecast - Яндекс Погода
#  getTopHeadlines - API новостной с пагинацей
#  getEverything - API новостной с пагинацей
#  suggestions_dadata_suggestBank - Dadata - сервис API

src_dict = {'getEverything':'C:/Users/kdenis/Documents/Work/OpenApiSpecParser/examples/NewsAPI.yaml'
            ,'suggestions_dadata_suggestBank':'C:/Users/kdenis/Documents/Work/OpenApiSpecParser/examples/suggestions.yml'
            ,'getTopHeadlines':'C:/Users/kdenis/Documents/Work/OpenApiSpecParser/examples/NewsAPI.yaml'
            ,'dataservice_accuweather_getCurrentConditions':'C:/Users/kdenis/Documents/Work/OpenApiSpecParser/examples/accuweather.yaml'}

Новостная:

Swift-банков

In [ ]:
input_data = {
    'getEverything':{'q':'RUSSIA'},
 'suggestions_dadata_suggestBank':[
    'SABRRUMM', 'VTBRRUMM', 'GAZPRUMM', 'ALFARUMM', 'MOSCRUMM',
    'RSBNRUMM', 'RUWCRUMM', 'ICBKRUMM', 'KOSKRUMM', 'PARNRUMM',
    'ABNYRUMM', 'CRYPRUMM', 'TICSRUMM', 'PSBZRUMM', 'TKRBRUMM',
    'JSNMRUMM', 'MIRBRUMM', 'ELSRUMMXXX', 'RNGBRUMM', 'IRONRUMM',
    'AVJSRUMM', 'ARESRUMM', 'ALILRUMM', 'ITRORU8Y', 'BOCSRUMM',
    'DOMRRUMM', 'FORTRUMM', 'GLBKRUMM', 'HDCBRUMM', 'KBKTRUMM',
    'KREMRUMM', 'LBRURUMM', 'MDMBRUMM', 'MEZHRUMM', 'MOPARUMM',
    'OLMDRUMM', 'ROYCRUMM', 'RZCBRUMM', 'SBERRUMM', 'SGBZRUMM',
    'SLAVRUMM', 'SOGZRUMM', 'TATKRUMM', 'TKBKRUMM', 'TKZLRUMM',
    'TKZVRUMM', 'TRNVRUMM', 'VEFKRUMM', 'VTBKRUMM', 'ZENIRUMM'
],
'getTopHeadlines':{'q':'RUSSIA'},
'dataservice_accuweather_getCurrentConditions':[1,2,3,45,66666,77777,9999,88]
}

Инициализция Парсера + Адаптера - различные варианты:

In [ ]:
parser = OASParser(src_dict[TOPIC_NAME])
entity  = parser.request.get(TOPIC_NAME)
pprint(entity)




In [ ]:
test = URESTAdapter(entity,'C:/Users/kdenis/mu_code/keys.json')
pprint(entity)


In [ ]:
result = test.execute(input_data.get(TOPIC_NAME))
pprint(result)


In [ ]:
resp  = parser.get_response(TOPIC_NAME)
pprint(resp)


Обработка спецификации и создание файла конфигурации